## 02_01_02_Outlier_Removal
### Context
Remove outliers based on Chebyshev inequation

### 0. Module initialization

In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime as DT
from datetime import timedelta
from copy import deepcopy as DC
from sklearn.neighbors import LocalOutlierFactor as LOF
from itertools import chain

sMyPath = r'C:\UCSD_ML_Capstone'
os.chdir(sMyPath)

### 1. Functions

In [2]:
def Outlier_Detection(Se, returnType = 'relative',Chebyshev_k=20,\
                      method = 'Chebyshev',n_neighbors = 20):
    
    lSe = DC(Se)
    if returnType == 'relative':
        rSe = np.log(lSe).diff()
    elif returnType == 'absolute':
        rSe = lSe.diff()
    else:
        raise ValueError('returnType should be absolute or relative.')
           
        
    if method == 'Chebyshev':
        ## calculate the deviation
        Dev = np.abs(rSe - rSe.mean())/rSe.std()
        ## remove the outliers
        remove_index = lSe[Dev>=Chebyshev_k].index 
    elif method == 'LOF':
        ## Local Outlier Factor model
        clf = LOF(n_neighbors=n_neighbors, n_jobs=4)
        ## Detect the outliers
        ifOutlier = clf.fit_predict(rSe.dropna().values.reshape(-1,1))
        ifOutlier = pd.Series(ifOutlier, index = range(1,len(ifOutlier)+1))
        remove_index = ifOutlier[ifOutlier==-1].index
        
    lSe[remove_index] = np.nan

    return([lSe, remove_index])  

### 2. Outlier Remove

#### 2.1. Path setting

In [3]:
datapath = sMyPath + r'\01_Input\01_02_RawData\Tiingo_Stock_daily'
savepath = sMyPath + r'\01_Input\01_03_ProcessedData\Tiingo_Stock_daily_Outlier_Removed'
AvaTickers_df = pd.read_csv(sMyPath + r"\01_Input\01_01_DataCodes\sNA_Select_Tickers.csv")
AvaTickers  = AvaTickers_df.Tickers.values

#### 2.2. Set the threshold of low volume

In [4]:
from scipy.stats import norm
P_sVol = 5/1000
Pert_sVol = norm.ppf(P_sVol)

TickerSummary=[]
OutliersSum = []

AvgSeries = pd.Series(np.empty(len(AvaTickers)),name = 'AvgSeries')
EstEndSeries = pd.Series(np.empty(len(AvaTickers)),name = 'EstEndSeries')

#### 2.3. Outlier Remove

In [5]:
## Time the loop
S_time = DT.now()
for iTicker in range(len(AvaTickers)):
    Loop_start = DT.now()

    ## Import the stock data
    Ticker = AvaTickers[iTicker]
    
    Sdata = pd.read_pickle(datapath + r"\\" + Ticker + "_daily.zip") 
    Sdata['date'] = [DT.strptime(x[:10], "%Y-%m-%d") for x in Sdata['date']]
    
    ## relatives transformation
    RelativeCols = Sdata[['open', 'high', 'low', 'close', 'adjOpen', 'adjHigh',\
                          'adjLow', 'adjClose']]
    RelativeCols = RelativeCols.apply(lambda x: Outlier_Detection(x,returnType = 'relative',\
                                              Chebyshev_k=20, method = 'Chebyshev')[0])
        
    Sdata[['open', 'high', 'low', 'close', 'adjOpen', 'adjHigh',\
                          'adjLow', 'adjClose']] = RelativeCols
        
    ## absolute transformation
    AbsoluteCols = Sdata[['volume', 'adjVolume']]
    AbsoluteCols = AbsoluteCols.apply(lambda x: Outlier_Detection(x,returnType = 'absolute',\
                                              Chebyshev_k=20, method = 'Chebyshev')[0])
        
    Sdata[['volume', 'adjVolume']] = AbsoluteCols
    Sdata = Sdata[Sdata['date']>=DT(2009,12,1)]

    ## Save processed data
    Sdata.to_pickle(savepath + r"\\" + Ticker + "_daily.zip")
    ## Check Na amount
    OutlierStats = Sdata.apply(lambda x: sum(np.isnan(x)) )
    OutlierStats.name = Ticker

    if len(OutliersSum)<=0:
        OutliersSum = OutlierStats
    else:
        OutliersSum = pd.concat([OutliersSum,OutlierStats],axis=1)
        
    ## Estimate finish time
    Loop_end = DT.now()
    LoopTime = Loop_end-Loop_start
    TotalTime = Loop_end-S_time
    AvgTime = (TotalTime)/(iTicker+1)
    EstEnd = DT.now() + (len(AvaTickers) - iTicker - 1)*AvgTime
    
    AvgSeries[iTicker] = AvgTime
    EstEndSeries[iTicker] = EstEnd
    print('Done No.'+ str(iTicker+1) + " " +Ticker+ ";\n" +
          "Loop time:" + str(LoopTime) + "\n" +
          "Total time:" + str(TotalTime) + "\n" +
          "Average time:" + str(AvgTime) + "\n" +
          "Estimate end:" + str(EstEnd) + '\n'  )


Done No.1 A;
Loop time:0:00:00.131662
Total time:0:00:00.132659
Average time:0:00:00.132659
Estimate end:2021-11-13 14:57:03.564781

Done No.2 APH;
Loop time:0:00:00.123670
Total time:0:00:00.258309
Average time:0:00:00.129154
Estimate end:2021-11-13 14:56:59.530527

Done No.3 ITRI;
Loop time:0:00:00.162565
Total time:0:00:00.421872
Average time:0:00:00.140624
Estimate end:2021-11-13 14:57:12.732496

Done No.4 MEI;
Loop time:0:00:00.128655
Total time:0:00:00.551524
Average time:0:00:00.137881
Estimate end:2021-11-13 14:57:09.575303

Done No.5 TER;
Loop time:0:00:00.135661
Total time:0:00:00.688193
Average time:0:00:00.137639
Estimate end:2021-11-13 14:57:09.296759

Done No.6 ROLL;
Loop time:0:00:00.130620
Total time:0:00:00.819811
Average time:0:00:00.136635
Estimate end:2021-11-13 14:57:08.141158

Done No.7 TKR;
Loop time:0:00:00.123696
Total time:0:00:00.943507
Average time:0:00:00.134787
Estimate end:2021-11-13 14:57:06.014107

Done No.8 AAON;
Loop time:0:00:00.144583
Total time:0:0


Done No.64 MGA;
Loop time:0:00:00.140623
Total time:0:00:08.101304
Average time:0:00:00.126583
Estimate end:2021-11-13 14:56:56.571297

Done No.65 MTOR;
Loop time:0:00:00.138628
Total time:0:00:08.240931
Average time:0:00:00.126784
Estimate end:2021-11-13 14:56:56.802627

Done No.66 SMP;
Loop time:0:00:00.113733
Total time:0:00:08.355662
Average time:0:00:00.126601
Estimate end:2021-11-13 14:56:56.592019

Done No.67 TEN;
Loop time:0:00:00.122673
Total time:0:00:08.479295
Average time:0:00:00.126557
Estimate end:2021-11-13 14:56:56.541355

Done No.68 TT;
Loop time:0:00:00.118685
Total time:0:00:08.598976
Average time:0:00:00.126456
Estimate end:2021-11-13 14:56:56.426093

Done No.69 ESE;
Loop time:0:00:00.133642
Total time:0:00:08.733615
Average time:0:00:00.126574
Estimate end:2021-11-13 14:56:56.560955

Done No.70 EXTR;
Loop time:0:00:00.126658
Total time:0:00:08.861272
Average time:0:00:00.126590
Estimate end:2021-11-13 14:56:56.579334

Done No.71 FLR;
Loop time:0:00:00.123698
Total

Done No.126 ETD;
Loop time:0:00:00.133603
Total time:0:00:16.532440
Average time:0:00:00.131210
Estimate end:2021-11-13 14:57:01.896962

Done No.127 LEG;
Loop time:0:00:00.158577
Total time:0:00:16.691017
Average time:0:00:00.131425
Estimate end:2021-11-13 14:57:02.144489

Done No.128 LZB;
Loop time:0:00:00.150596
Total time:0:00:16.841613
Average time:0:00:00.131575
Estimate end:2021-11-13 14:57:02.317110

Done No.129 MHK;
Loop time:0:00:00.122680
Total time:0:00:16.965290
Average time:0:00:00.131514
Estimate end:2021-11-13 14:57:02.246870

Done No.130 TPX;
Loop time:0:00:00.103725
Total time:0:00:17.070004
Average time:0:00:00.131308
Estimate end:2021-11-13 14:57:02.009744

Done No.131 AMWD;
Loop time:0:00:00.159572
Total time:0:00:17.230573
Average time:0:00:00.131531
Estimate end:2021-11-13 14:57:02.266465

Done No.132 GIL;
Loop time:0:00:00.125647
Total time:0:00:17.357234
Average time:0:00:00.131494
Estimate end:2021-11-13 14:57:02.223892

Done No.133 WOR;
Loop time:0:00:00.13264

Done No.186 UMC;
Loop time:0:00:00.111669
Total time:0:00:24.452259
Average time:0:00:00.131464
Estimate end:2021-11-13 14:57:02.189291

Done No.187 XLNX;
Loop time:0:00:00.127690
Total time:0:00:24.583940
Average time:0:00:00.131465
Estimate end:2021-11-13 14:57:02.190472

Done No.188 XPER;
Loop time:0:00:00.109706
Total time:0:00:24.694648
Average time:0:00:00.131355
Estimate end:2021-11-13 14:57:02.063785

Done No.189 ACLS;
Loop time:0:00:00.134630
Total time:0:00:24.829278
Average time:0:00:00.131372
Estimate end:2021-11-13 14:57:02.083414

Done No.190 ASML;
Loop time:0:00:00.125665
Total time:0:00:24.955940
Average time:0:00:00.131347
Estimate end:2021-11-13 14:57:02.054679

Done No.191 FSS;
Loop time:0:00:00.109708
Total time:0:00:25.066617
Average time:0:00:00.131239
Estimate end:2021-11-13 14:57:01.930329

Done No.192 GIC;
Loop time:0:00:00.131647
Total time:0:00:25.199261
Average time:0:00:00.131246
Estimate end:2021-11-13 14:57:01.938447

Done No.193 LRCX;
Loop time:0:00:00.1

Done No.246 ERIC;
Loop time:0:00:00.175502
Total time:0:00:32.589497
Average time:0:00:00.132478
Estimate end:2021-11-13 14:57:03.356359

Done No.247 FICO;
Loop time:0:00:00.151594
Total time:0:00:32.741091
Average time:0:00:00.132555
Estimate end:2021-11-13 14:57:03.445083

Done No.248 INTU;
Loop time:0:00:00.124657
Total time:0:00:32.866755
Average time:0:00:00.132527
Estimate end:2021-11-13 14:57:03.412908

Done No.249 MANH;
Loop time:0:00:00.120677
Total time:0:00:32.989427
Average time:0:00:00.132488
Estimate end:2021-11-13 14:57:03.367875

Done No.250 MSFT;
Loop time:0:00:00.120677
Total time:0:00:33.110104
Average time:0:00:00.132440
Estimate end:2021-11-13 14:57:03.312816

Done No.251 MSTR;
Loop time:0:00:00.123709
Total time:0:00:33.234810
Average time:0:00:00.132410
Estimate end:2021-11-13 14:57:03.278082

Done No.252 NATI;
Loop time:0:00:00.133639
Total time:0:00:33.369411
Average time:0:00:00.132418
Estimate end:2021-11-13 14:57:03.287465

Done No.253 NLOK;
Loop time:0:00:0

...


Done No.850 KSU;
Loop time:0:00:00.158575
Total time:0:02:08.945807
Average time:0:00:00.151701
Estimate end:2021-11-13 14:57:25.482080

Done No.851 NSC;
Loop time:0:00:00.169548
Total time:0:02:09.116352
Average time:0:00:00.151723
Estimate end:2021-11-13 14:57:25.507524

Done No.852 UNP;
Loop time:0:00:00.166559
Total time:0:02:09.283906
Average time:0:00:00.151742
Estimate end:2021-11-13 14:57:25.529036

Done No.853 AMOV;
Loop time:0:00:00.143614
Total time:0:02:09.428517
Average time:0:00:00.151733
Estimate end:2021-11-13 14:57:25.519223

Done No.854 AMX;
Loop time:0:00:00.197473
Total time:0:02:09.625990
Average time:0:00:00.151787
Estimate end:2021-11-13 14:57:25.581001

Done No.855 IDCC;
Loop time:0:00:00.163561
Total time:0:02:09.789551
Average time:0:00:00.151801
Estimate end:2021-11-13 14:57:25.596919

Done No.856 IDT;
Loop time:0:00:00.166554
Total time:0:02:09.957102
Average time:0:00:00.151819
Estimate end:2021-11-13 14:57:25.617979

Done No.857 INFN;
Loop time:0:00:00.13

Done No.911 PEG;
Loop time:0:00:00.182515
Total time:0:02:19.134561
Average time:0:00:00.152727
Estimate end:2021-11-13 14:57:26.663313

Done No.912 PNM;
Loop time:0:00:00.172539
Total time:0:02:19.308096
Average time:0:00:00.152750
Estimate end:2021-11-13 14:57:26.689618

Done No.913 PNW;
Loop time:0:00:00.190488
Total time:0:02:19.499582
Average time:0:00:00.152793
Estimate end:2021-11-13 14:57:26.738588

Done No.914 POR;
Loop time:0:00:00.143617
Total time:0:02:19.645194
Average time:0:00:00.152785
Estimate end:2021-11-13 14:57:26.729511

Done No.915 PPL;
Loop time:0:00:00.159571
Total time:0:02:19.805763
Average time:0:00:00.152793
Estimate end:2021-11-13 14:57:26.739183

Done No.916 OKE;
Loop time:0:00:00.150598
Total time:0:02:19.957359
Average time:0:00:00.152792
Estimate end:2021-11-13 14:57:26.737751

Done No.917 SWX;
Loop time:0:00:00.171541
Total time:0:02:20.129897
Average time:0:00:00.152813
Estimate end:2021-11-13 14:57:26.762411

Done No.918 DCP;
Loop time:0:00:00.131647

Done No.971 INSM;
Loop time:0:00:00.184507
Total time:0:02:29.080959
Average time:0:00:00.153533
Estimate end:2021-11-13 14:57:27.591171

Done No.972 INVA;
Loop time:0:00:00.155584
Total time:0:02:29.237540
Average time:0:00:00.153537
Estimate end:2021-11-13 14:57:27.594935

Done No.973 IONS;
Loop time:0:00:00.159573
Total time:0:02:29.398110
Average time:0:00:00.153544
Estimate end:2021-11-13 14:57:27.603214

Done No.974 JNJ;
Loop time:0:00:00.165558
Total time:0:02:29.564665
Average time:0:00:00.153557
Estimate end:2021-11-13 14:57:27.618526

Done No.975 LGND;
Loop time:0:00:00.170546
Total time:0:02:29.736208
Average time:0:00:00.153576
Estimate end:2021-11-13 14:57:27.639856

Done No.976 LLY;
Loop time:0:00:00.179518
Total time:0:02:29.915726
Average time:0:00:00.153602
Estimate end:2021-11-13 14:57:27.670348

Done No.977 MDGL;
Loop time:0:00:00.151596
Total time:0:02:30.068320
Average time:0:00:00.153601
Estimate end:2021-11-13 14:57:27.669166

Done No.978 MEIP;
Loop time:0:00:00.

Done No.1032 CYH;
Loop time:0:00:00.160570
Total time:0:02:39.842179
Average time:0:00:00.154886
Estimate end:2021-11-13 14:57:29.147885

Done No.1033 ENSG;
Loop time:0:00:00.147605
Total time:0:02:39.990782
Average time:0:00:00.154880
Estimate end:2021-11-13 14:57:29.140894

Done No.1034 MGLN;
Loop time:0:00:00.144613
Total time:0:02:40.135395
Average time:0:00:00.154870
Estimate end:2021-11-13 14:57:29.129457

Done No.1035 NHC;
Loop time:0:00:00.180520
Total time:0:02:40.316912
Average time:0:00:00.154896
Estimate end:2021-11-13 14:57:29.159120

Done No.1036 SEM;
Loop time:0:00:00.116687
Total time:0:02:40.433599
Average time:0:00:00.154859
Estimate end:2021-11-13 14:57:29.116656

Done No.1037 THC;
Loop time:0:00:00.189490
Total time:0:02:40.624088
Average time:0:00:00.154893
Estimate end:2021-11-13 14:57:29.156162

Done No.1038 UHS;
Loop time:0:00:00.166555
Total time:0:02:40.791641
Average time:0:00:00.154905
Estimate end:2021-11-13 14:57:29.170178

Done No.1039 ALGN;
Loop time:0:0

Done No.1092 KOF;
Loop time:0:00:00.160571
Total time:0:02:50.635315
Average time:0:00:00.156259
Estimate end:2021-11-13 14:57:30.728868

Done No.1093 PEP;
Loop time:0:00:00.137633
Total time:0:02:50.774942
Average time:0:00:00.156244
Estimate end:2021-11-13 14:57:30.711366

Done No.1094 SAM;
Loop time:0:00:00.155583
Total time:0:02:50.931523
Average time:0:00:00.156245
Estimate end:2021-11-13 14:57:30.711760

Done No.1095 TAP;
Loop time:0:00:00.142619
Total time:0:02:51.075139
Average time:0:00:00.156233
Estimate end:2021-11-13 14:57:30.698459

Done No.1096 ELY;
Loop time:0:00:00.157594
Total time:0:02:51.233741
Average time:0:00:00.156235
Estimate end:2021-11-13 14:57:30.700938

Done No.1097 HAS;
Loop time:0:00:00.153590
Total time:0:02:51.388302
Average time:0:00:00.156234
Estimate end:2021-11-13 14:57:30.699210

Done No.1098 MAT;
Loop time:0:00:00.205453
Total time:0:02:51.594753
Average time:0:00:00.156279
Estimate end:2021-11-13 14:57:30.751812

Done No.1099 PII;
Loop time:0:00:0